In [ ]:
!pip install datasets pandas openai pymongo 

In [6]:
from datasets import load_dataset
import pandas as pd

# <https://huggingface.co/datasets/AIatMongoDB/embedded_movies>
dataset = load_dataset("AIatMongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset['train'])

dataset_df.head(5)

,fullplot,num_mflix_comments,type,poster,rated,plot,languages,countries,plot_embedding,metacritic,genres,writers,awards,directors,cast,runtime,title,imdb
0,Young Pauline is left a lot of money when her ...,0,movie,https://m.media-amazon.com/images/M/MV5BMzgxOD...,None,Young Pauline is left a lot of money when her ...,[English],[USA],"[0.00072939653, -0.026834568, 0.013515796, -0....",NaN,[Action],"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Louis J. Gasnier, Donald MacKenzie]","[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",199.0,The Perils of Pauline,"{'id': 4465, 'rating': 7.6, 'votes': 744}"
1,As a penniless man worries about how he will m...,0,movie,https://m.media-amazon.com/images/M/MV5BNzE1OW...,TV-G,A penniless young man tries to save an heiress...,[English],[USA],"[-0.022837115, -0.022941574, 0.014937485, -0.0...",NaN,"[Comedy, Short, Action]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Alfred J. Goulding, Hal Roach]","[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",22.0,From Hand to Mouth,"{'id': 10146, 'rating': 7.0, 'votes': 639}"
2,"Michael ""Beau"" Geste leaves England in disgrac...",0,movie,None,None,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[USA],"[0.00023330493, -0.028511643, 0.014653289, -0....",NaN,"[Action, Adventure, Drama]","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Herbert Brenon],"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",101.0,Beau Geste,"{'id': 16634, 'rating': 6.9, 'votes': 222}"
3,A nobleman vows to avenge the death of his fat...,1,movie,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"Seeking revenge, an athletic young man joins t...",None,[USA],"[-0.005927917, -0.033394486, 0.0015323418, -0....",NaN,"[Adventure, Action]","[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Albert Parker],"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",88.0,The Black Pirate,"{'id': 16654, 'rating': 7.2, 'votes': 1146}"
4,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",0,movie,https://m.media-amazon.com/images/M/MV5BMTcxMT...,PASSED,An irresponsible young millionaire changes his...,[English],[USA],"[-0.0059373598, -0.026604708, -0.0070914757, -...",NaN,"[Action, Comedy, Romance]","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",[Sam Taylor],"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",58.0,For Heaven's Sake,"{'id': 16895, 'rating': 7.6, 'votes': 918}"


In [7]:
# Remove data point where plot column is missing
dataset_df = dataset_df.dropna(subset=['plot'])
print("\\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with the new OpenAI embedding Model "text-embedding-3-small"
dataset_df = dataset_df.drop(columns=['plot_embedding'])
dataset_df.head(5)

\nNumber of missing values in each column after removal:
fullplot               21
num_mflix_comments      0
type                    0
poster                 78
rated                 284
plot                    0
languages               1
countries               0
plot_embedding          1
metacritic            903
genres                  0
writers                13
awards                  0
directors              13
cast                    1
runtime                14
title                   0
imdb                    0
dtype: int64


,fullplot,num_mflix_comments,type,poster,rated,plot,languages,countries,metacritic,genres,writers,awards,directors,cast,runtime,title,imdb
0,Young Pauline is left a lot of money when her ...,0,movie,https://m.media-amazon.com/images/M/MV5BMzgxOD...,None,Young Pauline is left a lot of money when her ...,[English],[USA],NaN,[Action],"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Louis J. Gasnier, Donald MacKenzie]","[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",199.0,The Perils of Pauline,"{'id': 4465, 'rating': 7.6, 'votes': 744}"
1,As a penniless man worries about how he will m...,0,movie,https://m.media-amazon.com/images/M/MV5BNzE1OW...,TV-G,A penniless young man tries to save an heiress...,[English],[USA],NaN,"[Comedy, Short, Action]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","[Alfred J. Goulding, Hal Roach]","[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",22.0,From Hand to Mouth,"{'id': 10146, 'rating': 7.0, 'votes': 639}"
2,"Michael ""Beau"" Geste leaves England in disgrac...",0,movie,None,None,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[USA],NaN,"[Action, Adventure, Drama]","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Herbert Brenon],"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",101.0,Beau Geste,"{'id': 16634, 'rating': 6.9, 'votes': 222}"
3,A nobleman vows to avenge the death of his fat...,1,movie,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"Seeking revenge, an athletic young man joins t...",None,[USA],NaN,"[Adventure, Action]","[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Albert Parker],"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",88.0,The Black Pirate,"{'id': 16654, 'rating': 7.2, 'votes': 1146}"
4,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",0,movie,https://m.media-amazon.com/images/M/MV5BMTcxMT...,PASSED,An irresponsible young millionaire changes his...,[English],[USA],NaN,"[Action, Comedy, Romance]","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",[Sam Taylor],"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",58.0,For Heaven's Sake,"{'id': 16895, 'rating': 7.6, 'votes': 918}"


In [8]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API")

EMBEDDING_MODEL = "text-embedding-3-small"

def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

dataset_df["plot_embedding_optimised"] = dataset_df['plot'].apply(get_embedding)

dataset_df.head()

Error in get_embedding: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-Xw8SOSR05Q01XWfY4FVztnB1 on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}


KeyboardInterrupt: 

In [ ]:
import pymongo

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = os.getenv("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client['movies']
collection = db['movie_collection']

documents = dataset_df.to_dict('records')
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

In [ ]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "plot_embedding_optimised",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 5  # Return top 5 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "plot_embedding_opitimzed": 0,  # Exclude the plot_embedding_opitimzed field
                "plot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1, # Include the genres field
                "score": {
                    "$meta": "vectorSearchScore"  # Include the search score
                }
            }
        }
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)


In [ ]:
def handle_user_query(query, collection):

  get_knowledge = vector_search(query, collection)

  search_result = ''
  for result in get_knowledge:
      search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('plot', 'N/A')}\\n"

  completion = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a movie recommendation system."},
          {"role": "user", "content": "Answer this user query: " + query + " with the following context: " + search_result}
      ]
  )

  return (completion.choices[0].message.content), search_result

Conduct query with retrieval of sources
query = "What is the best romantic movie to watch?"
response, source_information = handle_user_query(query, collection)

print(f"Response: {response}")
print(f"Source Information: \\n{source_information}")